# Semplice Linear Regression con dei Dati Sintetici

## Usa la versione giusta di TensorFlow

La seguente riga di codice assicura che Colab eseguirà TensorFlow 2.X, corrispondente alla versione più recente di TensorFlow:

In [ ]:
#@title Run this Colab on TensorFlow 2.x
%tensorflow_version 2.x

## Import di moduli necessari



In [ ]:
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt

#Definizione delle funzioni che costruiscono e allenano un modello

Il seguente codice definisce due funzioni:

* `build_model(my_learning_rate)`, che costruisce un modello vuoto
* `train_model(model, feature, label, epochs)`, che allena il modello a partire dai dati (feature e label) che gli passi.

Dato che non è necessario per adesso capire come il codice per costruire il modello è stato sviluppato, lo abbiamo nascosto. Possiamo però fare doppio click per esplorarlo.


In [ ]:
#@title Definizione delle funzioni che costruiscono ed allenano il modello

def build_model(my_learning_rate):
  """Crea e compila un semplice modello di regressione lineare."""
  # I modelli più semplici di tf.keras sono sequenziali.
  # Un modello sequenziale contiene uno o più livelli.
  model = tf.keras.models.Sequential()

  # Descrive la topografia del modello.
  # La topografia di un semplice modello di regressione lineare
  # è un singolo nodo in un singolo livello
  model.add(tf.keras.layers.Dense(units=1, 
                                  input_shape=(1,)))

  # Compila la topografia del modello in codice che TensorFlow
  # può eseguire efficientemente. Configura l'allenamento per
  # minimizzare l'errore quadratico medio del modello
  model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=my_learning_rate),
                loss="mean_squared_error",
                metrics=[tf.keras.metrics.RootMeanSquaredError()])

  return model           


def train_model(model, feature, label, epochs, batch_size):
  """Allena il modello dandogli in pasto i dati."""

  # Dai in pasto al modello i valori delle features e i valori delle lables.
  # Il modello si allenerà per il numero specificato di epoche, imparando
  # gradualmente come i valori delle features sono in relazione con
  # i valori delle lables.
  history = model.fit(x=feature,
                      y=label,
                      batch_size=batch_size,
                      epochs=epochs)

  # Raccoglie il weight e bias del modello allenato
  trained_weight = model.get_weights()[0]
  trained_bias = model.get_weights()[1]

  # La lista delle epoche è salvata separatamente rispetto 
  # il resto della history.
  epochs = history.epoch
  
  # Raccoglie la history (una snapshot) di ogni epoca.
  hist = pd.DataFrame(history.history)

  # Raccoglie specificamente l'errore quadratico medio 
  # del modello ad ogni epoca.
  rmse = hist["root_mean_squared_error"]

  return trained_weight, trained_bias, epochs, rmse

print("Defined create_model and train_model")

## Definizione delle funzioni per il plotting

Usiamo una libreria molto conosciuta chiamata  [Matplotlib](https://developers.google.com/machine-learning/glossary/#matplotlib) per creare i seguenti due grafici:

* Un grafico dei valori delle features vs. i valori delle lables, ed una linea che mostra l'output del modello allenato.
* Una [curva di loss](https://developers.google.com/machine-learning/glossary/#loss_curve).


In [ ]:
#@title Definizione delle funzioni di plotting
def plot_the_model(trained_weight, trained_bias, feature, label):
  """Disegna il modello allenato rispetto feature e le label usate per il training."""

  # Diamo un nome agli assi.
  plt.xlabel("feature")
  plt.ylabel("label")

  # Disegna i valori delle features vs. i valori delle lables
  plt.scatter(feature, label)

  # Crea una linea rossa rappresentante il modello. La linea rossa
  # ha origine alle coordinate (x0, y0) e termina alle coordinate (x1, y1).
  x0 = 0
  y0 = trained_bias
  x1 = feature[-1]
  y1 = trained_bias + (trained_weight * x1)
  plt.plot([x0, x1], [y0, y1], c='r')

  # Renderizza il grafico a dispersione e la linea rossa
  plt.show()

def plot_the_loss_curve(epochs, rmse):
  """Disegna la curva di loss, che mostra la loss vs. le epoche."""

  plt.figure()
  plt.xlabel("Epoch")
  plt.ylabel("Root Mean Squared Error")

  plt.plot(epochs, rmse, label="Loss")
  plt.legend()
  plt.ylim([rmse.min()*0.97, rmse.max()])
  plt.show()

print("Defined the plot_the_model and plot_the_loss_curve functions.")


## Definizione deld ataset

Il dataset è costituito da 12 dati.


In [ ]:
my_feature = ([1.0, 2.0,  3.0,  4.0,  5.0,  6.0,  7.0,  8.0,  9.0, 10.0, 11.0, 12.0])
my_label   = ([5.0, 8.8,  9.6, 14.2, 18.8, 19.5, 21.4, 26.8, 28.9, 32.0, 33.8, 38.2])

## Specifica gli iperparametri

Gli iperparametri in questo Colab sono:

  * [learning rate](https://developers.google.com/machine-learning/glossary/#learning_rate)
  * [epoche](https://developers.google.com/machine-learning/glossary/#epoch)
  * [batch_size](https://developers.google.com/machine-learning/glossary/#batch_size)

Il codice seguente inizializza questi iperparametri e poi invoca le funzioni per costruire ed allenare il modello.

In [ ]:
learning_rate=0.01
epochs=10
my_batch_size=12

my_model = build_model(learning_rate)
trained_weight, trained_bias, epochs, rmse = train_model(my_model, my_feature, 
                                                         my_label, epochs,
                                                         my_batch_size)
plot_the_model(trained_weight, trained_bias, my_feature, my_label)
plot_the_loss_curve(epochs, rmse)

## Task 1: Esamina il grafico

Esamina il grafo sovrastante. I punti blu rappresentano i dati; la linea rossa rappresenta l'outpu del modello allenato. Idealmente, la linea rossa dovrebbe allinearsi armoniosamente con i punti blu. Lo fa? No.

Esamina il grafo sottostante, che mostra la curva di loss. Nota come la curva di loss decresce ma non si appiattisce, che è un segno che il modello non si è allenato a sufficienza.

Examine the bottom graph, which shows the loss curve. Notice that the loss curve decreases but doesn't flatten out, which is a sign that the model hasn't trained sufficiently.

## Task 2: Aumenta il numero delle epoche

La loss dovrebbe decrementare in modo costante, dapprima bruscamente, e poi più lentamente. Eventualmente, la loss dovrebbe rimanere stabile (con pendenza zero o quasi zero), che indica che l'allenamento ha raggiunto la [convergenza](http://developers.google.com/machine-learning/glossary/#convergence).

In Task 1, la loss non ha raggiunto la convergenza. Una possibile soluzione è allenare il modello per più epoche. Il tuo task è aumentare il numero delle epoche sufficientemente per far si che il modello converga. Tuttavia, è inefficiente allenare il modello oltrepassata la convergenza, quindi non impostare semplicemente un numero di epoche arbitrariamente molto alto.

Esamina la curva di loss. Il modello converge?


In [ ]:
learning_rate=0.01
epochs= 450  # Replace ? with an integer.
my_batch_size=12

my_model = build_model(learning_rate)
trained_weight, trained_bias, epochs, rmse = train_model(my_model, my_feature, 
                                                        my_label, epochs,
                                                        my_batch_size)
plot_the_model(trained_weight, trained_bias, my_feature, my_label)
plot_the_loss_curve(epochs, rmse)

## Task 3: Aumenta il Learning rate

Nel Task 2, abbiamo incrementato i l numero di epoche per far si che il modello raggiungesse la convergenza. A volte, possiamo raggiungere la convergenza molto più velocemente aumentando il learning rate. Tuttavia, impostando un learning rate troppo altro, spesso rende il modello impossibilitato a convergere. Nel Task 3, abbiamo intenzionalmente impostato il learning rate troppo alto.


In [ ]:
# Aumenta il learning rate e diminuisci il numero delle epoche.
learning_rate=100  
epochs=500  

# Iparametri perfetti invece sono
# learning_rate=0.14
# epochs=70

my_model = build_model(learning_rate)
trained_weight, trained_bias, epochs, rmse = train_model(my_model, my_feature, 
                                                         my_label, epochs,
                                                         my_batch_size)
plot_the_model(trained_weight, trained_bias, my_feature, my_label)
plot_the_loss_curve(epochs, rmse)

Il modello risultante è terribile; la linea rossa non si allinea con i punti blu. Inoltre, la curva di loss oscilla come una [montagna russa](https://www.wikipedia.org/wiki/Roller_coaster). Una curva di loss che oscilla suggerisce fortemente che il learning rate è troppo alto.

## Task 5: Regola la batch size

Il sistema ricalcola il valore di loss del modello regolando i pesi e dell'intercetta dopo ogni **iterazione**. Ogni iterazione è il frangente in cui il sistema processa una batch. Per esempio, se la **batch size** è 6, allora il sistema ricalcola il valore della loss del modello regolando i pesi del modello e l'intercetta dopo ogni 6 dati.

Una **epoca** è il frangente sufficiente di ogni iterazione per processare ogni dato presente nel dataset. Per esempio, se la batch size è 12, allora ogni epoca dura una iterazione. Tuttavia, se la batch size è 6, allora ogni epoca consuma due iterazioni.

Si potrebbe essere tentati dall'impostare la batch size uguale al numero dei dati nel dataset (12, in questo caso). Tuttavia, il modello potrebbe allenarsi più velocemente su batch più piccole. Al contrario, su delle batch size molto piccole, potrebbe non contenere abbastanza informazioni per aiutare il modello a convergere.

Esperimenta con la `batch_size`nel seguente codice. Quale è il più piccolo intero che è possibile dare a `batch_size` ed avere ancora il modello che raggiunge la convergenza in un centinaio di epoche? 


In [ ]:
learning_rate=0.05
epochs=125
my_batch_size= 1  # Replace ? with an integer. Settando 1 praticamente ho realizzato la Stocastic Gradient Descent la quale usa solo 1 dato.

my_model = build_model(learning_rate)
trained_weight, trained_bias, epochs, rmse = train_model(my_model, my_feature, 
                                                        my_label, epochs,
                                                        my_batch_size)
plot_the_model(trained_weight, trained_bias, my_feature, my_label)
plot_the_loss_curve(epochs, rmse)

## Ricapitolo della calibrazione degli iperparametri

La maggior parte dei problemi di machine learning richiedono molto tuning degli iperparametri. Sfortunatamente, non possiamo fornire delle regole concrete per la calibrazione di ogni modello. Abbassare il learning rate può aiutare un modello a convergere efficientemente ma potrebbe rendere un altro modello troppo lento a raggiungere la convergenza. Devi sperimentare per trovare il miglior insieme di iperparametri per il tuo dataset. Detto questo, ecco delle linee guida:

* La loss dovrebbe decrescere in modo costante, dapprima bruscamente, e poi man mano più gradualmente finchè la pendenza della curva raggiunge o si avvicina allo zero.
* Se la loss non converge, allena il modello per più epoche.
* Se la loss decrementa troppo lentamente, aumenta il learning rate. Nota che impostando il learning rate troppo alto potrebbe prevenire la loss dal convergere.
* Se la loss varia in modo bizzarro (cioè, la loss salta in giro) decrementare il learning rate.
* Diminuire il learning rate mentre si incrementa il numero di epoche o la batch size è spesso una buona combinazione.
* Impostare la batch size ad un numero *molto* basso potrebbe anche causare instabilità. Per prima cosa, prova un valore per la batch size grande. Poi, decrementalo finchè non vedi un peggioramento. 

* Per i dataset del mondo-reale i quali contengono milioni o miliardi di dati, l'intero dataset potrebbe addirittura non entrare in memoria. In questi casi, avrai bisogno di ridurre la batch size per permettere alla batch di entrare in memoria.

Ricorda: La combinazione ideale di iperparametri è dipendente dai dati, quindi devi sempre sperimentare e verificare.
